In [ ]:
#importando librerias
import pandas as pd
import numpy as np
import seaborn as sns
from Database import Airbnbs, Hosts, Airbnb_Details, Neighbourhoods, creating_engine, creating_session, closing_session

In [ ]:
#Creating engine
engine1 = creating_engine()

#Creating session
session1 = creating_session(engine1)

## Proceso de Exploración

### Leemos el archivo como un dataframe usando pandas

In [ ]:
df_airbnb=pd.read_csv("Airbnb_Open_Data.csv",  na_values=[''])
df_airbnb.head(2)

### Se hacen ajustes a ciertas columnas con el fin de poder subir el csv a la base de datos 

In [ ]:
#Usamos regrex para evitar los caracteres especiales y númericos
df_airbnb["NAME"] = df_airbnb["NAME"].str.replace(r'[^a-zA-Z\s]', '', regex=True)
df_airbnb["house_rules"] = df_airbnb["house_rules"].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_airbnb["host name"] = df_airbnb["host name"].str.replace(r'[^a-zA-Z\s]', '', regex=True)

#Dropeamos aquellas columnas que no aportan un valor signficante al código 
df_airbnb.drop(["license", "country", "country code"], axis=1, inplace=True)

#Quitamos los valores nulos del host name
df_airbnb["host name"].fillna("no provided") 

#Admitimos solo valores positivos para la columna minimum nights
df_airbnb["minimum nights"] = df_airbnb["minimum nights"].abs()



df_airbnb.to_csv("import.csv", encoding="utf-8", index=False)

In [ ]:
df_airbnb.to_sql("airbnbs", con=engine1, if_exists="replace", index=False)
print("Succesfully Created")

### ¿Qué tipo de dato son las variables del conjunto de datos?

In [ ]:
df_airbnb.dtypes

### ¿Cuántas variables de cada tipo de dato tenemos en el conjunto de datos?

In [ ]:
(
    df_airbnb
    .dtypes
    .value_counts()
)

###  ¿Cuales son las dimensiones del Dataset?

In [ ]:
df_airbnb.shape

### ¿Existen valores nulos explicitos en el conjunto de datos?

In [ ]:
df_airbnb.isnull().any()

### De tener observaciones con valores nulos, ¿cuántas tenemos por cada variable?

In [ ]:
(
    df_airbnb
    .isnull()
    .sum()
    .sort_values(ascending=False)
)


### ¿Cuál es la proporción de valores nulos por cada variable? 

In [ ]:
(
    df_airbnb
    .isnull()
    .melt(value_name='missing')
    .pipe(
        lambda df: (
            sns.displot(
                data=df,
                y='variable',
                hue='missing',
                multiple='fill',
                aspect=2
            )
        )
    )
)

### ¿Cuántos valores nulos tenemos en total en el conjunto de datos?

In [ ]:
(
    df_airbnb
    .isnull()
    .sum()
    .sum()
)

## Proceso de Limpieza 

#### Estandarizamos los nombres

In [ ]:
new_column_names = [x.lower().replace(" ", "_") for x in df_airbnb.columns]
df_airbnb.columns = new_column_names

#### Dado que el atributo de "host_identity_verified" cuenta con valores nulos, vamos a llenar dichos valores con unverified

In [ ]:
df_airbnb["host_identity_verified"].fillna("unverified", inplace=True)
df_airbnb

#### Es necesario eliminar los símbolos de dólar de las columnas "price" y "service_fee"

In [ ]:
df_airbnb["price"] = df_airbnb["price"].str.replace('$', '').str.replace(',', '').str.strip()
df_airbnb["service_fee"] = df_airbnb["service_fee"].str.replace('$', '').str.replace(',', '').str.strip()

# Convertir las columnas "price" y "service_fee" a valores flotantes
df_airbnb["price"] = df_airbnb["price"].astype(float)
df_airbnb["service_fee"] = df_airbnb["service_fee"].astype(float)

#Verificamos que no haya ningún registro en el que el service fee sea mayor al precio
counter = df_airbnb["price"] < df_airbnb["service_fee"]
count_greater_service_fee = counter.sum()
count_greater_service_fee

#### Limpia los valores no finitos en la columna "Construction year"

In [ ]:
df_airbnb["construction_year"] = df_airbnb["construction_year"].replace([np.inf, -np.inf], np.nan)
df_airbnb["construction_year"] = df_airbnb["construction_year"].fillna(0)  # Rellenar valores nulos con 0 o el valor 

#### Cambiamos la columna "Construction year" de tipo float64 a int32, ya que los años son enteros.

In [ ]:

df_airbnb["construction_year"] = df_airbnb["construction_year"].astype(int)

#### Reemplaza los valores nulos por 0 en las columnas "last review" y "reviews per month". Cuando la columna number of reviews es 0.

In [ ]:
filtered_df = df_airbnb[df_airbnb["number_of_reviews"] == 0]
# Muestra los registros filtrados
print(filtered_df)

# Cuenta los valores nulos en la columna "last_review"
null_last_review = filtered_df["last_review"].isnull().sum()
# Cuenta los valores nulos en la columna "reviews_per_month"
null_reviews_per_month = filtered_df["reviews_per_month"].isnull().sum()

# Muestra los resultados
print("Registros nulos en last review:", null_last_review)
print("Registros nulos en reviews per month:", null_reviews_per_month)

In [ ]:
df_airbnb.loc[df_airbnb["number_of_reviews"] == 0, "last_review"] = 0
df_airbnb.loc[df_airbnb["number_of_reviews"] == 0, "reviews_per_month"] = 0

#### Remplazar los nulos de la columna house_rules 

In [ ]:
df_airbnb["house_rules"] = df_airbnb["house_rules"].fillna("No se Especificaron Las Reglas")

### Creación de las dimensiones 

In [ ]:
neighbourhood_table = df_airbnb[["neighbourhood_group", "neighbourhood", "lat", "long"]]
neighbourhood_table.head(2)

In [ ]:
host_table = df_airbnb[["host_id", "host_name", "host_identity_verified"]]
host_table.head(2)

In [ ]:
airbnb_detail = df_airbnb[
    ["id", "name", "instant_bookable", 
     "cancellation_policy", "room_type", "construction_year", 
     "price", "service_fee", "minimum_nights", "number_of_reviews", 
     "last_review", "reviews_per_month", "review_rate_number", "calculated_host_listings_count", 
     "availability_365", "house_rules"]]

airbnb_detail.head(2)

### Transformaciones a la dimensión de Neighbourhoods

#### Agrupamos con el fin de no tener datos redundantes, calculando además promedio de latitud y longitud

In [ ]:
neighbourhood_table = neighbourhood_table.groupby(["neighbourhood_group", "neighbourhood"]).agg({
    "lat": "mean",
    "long": "mean"
}).reset_index()

#Creamos ID y reorganizamos columnas
neighbourhood_table["neighbourhood_id"] = range(1, len(neighbourhood_table) + 1)
column_order = ["neighbourhood_id", "neighbourhood_group", "neighbourhood", "lat", "long"]
neighbourhood_table = neighbourhood_table[column_order]

neighbourhood_table.head(2)

### Proceso de limpieza a nuestra tabla de hecho (df_airbnb)

In [ ]:
#Delete the following columns since we already set up diferent tables to save all the information related with neighbourhoods and hosts
df_airbnb.drop(
         ["id", "name", "instant_bookable", 
     "cancellation_policy", "room_type", "construction_year", 
     "price", "service_fee", "minimum_nights", "number_of_reviews", 
     "last_review", "reviews_per_month", "review_rate_number", 
     "calculated_host_listings_count", 
     "availability_365", "house_rules", "host_name", 
     "host_identity_verified", "lat", "long"], axis=1, inplace=True)

#### Se cambian las columnas neighbourhood_group y neighbourhood por un identificador númerico

In [ ]:
def get_neighbourhood_id(row):
    mask = (neighbourhood_table["neighbourhood_group"] == row["neighbourhood_group"]) & (neighbourhood_table["neighbourhood"] == row["neighbourhood"])
    matched_row = neighbourhood_table[mask]
    if not matched_row.empty:
        return matched_row["neighbourhood_id"].iloc[0]
    else:
        return None

# Aplicar la función para obtener "neighbourhood_id"
df_airbnb["neighbourhood_id"] = df_airbnb.apply(get_neighbourhood_id, axis=1)

#### Una vez asignados los IDs de forma númerica podemos prescindir de las columnas iniciales

In [ ]:
df_airbnb.drop(["neighbourhood_group", "neighbourhood"], axis=1, inplace=True)
df_airbnb

## Proceso de Carga

In [ ]:
#Creating engine
engine1 = creating_engine()

#Creating session
session1 = creating_session(engine1)